# Random-number simulations using NumPy

This notebook will illustrate the mechanics of performing random-number simulations using the mechanics of NumPy we have already seen.  We will do two examples -- one sampling from a continuous distribution, one from a discrete distribution -- and see how to make a lot of headway with very few lines of code.

The details of the simulation exercises can be found in the accompanying file `orf409_two_simulations.pdf`.

In the process, we'll encounter some useful functions from the `numpy.random` module.  The following link has a summary of the routines and functions found in `numpy.random`:

[https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html)

We will try to browse this "toolkit" for tools that we can leverge to do our job, in order to avoid reinventing the wheel.  When possible, leverage what you have rather than writing things from scratch that might be buggy.

**NOTE:**  More recent versions of NumPy have adopted a somewhat different interface for generating random numbers than what is detailed in this notebook.  The main reason for those enhancements is performance and not user-friendliness.  So since the older interface is still supported (albeit deprecated), that is what you see in this notebook.  Unless you're running a crazy simulation on a cluster, you're better off with what's below, especially if you have limted prior experience with NumPy and Python.

## Setup -- importing numpy & seeding the random number generator

Before we start, we need to import numpy into this notebook.

In [ ]:
import numpy as np

Before each example, we should also set up a **seed** for the random number generator...

Computers don't *actually* generate random numbers.  Rather, they generate pseudo-random numbers that, in principle, can be reproduced exactly *provided* you prime the pseudorandom-number generator with the same **seed** each time.

Just so we are all getting the same results in our various notebooks, let's all use the same seed and set it up at the start of each problem (using a seed is good habit to get into, for personal debugging purposes).  The easiest thing to use as a seed is any biggish integer.  We'll do this at the start of each example problem.

## Simulation 1

This problem asks us to draw random values uniformly from the interval $[0,1)$ until the running sum exceeds 1.  Call $N$ the number of values we had to generate until we reached a sum above 1. Repeat this experiment 100 total times and average all the values of $N$ to get an estimate of $E[N]$.

Then repeat this experiment 1K times and 10K times and extrapolate to get an increasingly robust estimate for $E[N]$.

### Strategy
How should we implement this in code?

One strategy is to write a loop -- generate one random number at a time, keep a running sum, record how many values it takes to exceed 1, and then start the loop anew, looping over *that* inner loop 100 times (or 1K times or 10K times).

This would work, but it would be a lot of code.  Instead, we can lighten our coding burden by leveraging some obvious (and a few less obvious) features of numpy...

`np.random.random()` can be a given a single scalar argument (indicating the number of values to choose) or a tuple (indicating a shape if you want an array with more than 1 dimension).  Now, realistically, how many values do suspect we'd need to draw before we got a sum that exceeded 1? I'd say 1000 is probably fairly safe (and if it turns out not to be, we can come back and increase that value).

So why don't we just generate a 2D array of random values:  10K rows (each row is one "run" of the experiment), and 1000 columns (i.e. 1000 random values drawn in each experiment). Then, along each row, we can perform a `cumsum()` (which gives a running total) and pick out in each row the index where the running total has exceeded 1 (given how `cumsum()` works, i.e. that the entry in the result with index $k$ is the result of adding the first $k+1$ numbers in the original array, we'll need to add 1 to our index results to get the right value of $N$ in each experiment).

What we'll have by the end is an array of 10K $N$ values.  To estimate $E[N]$ using only 100 values, we can just slice, and average the first 100.  Ditto for the first 1000.

It turns out that all of this can be done in very few lines of numpy, with no loops anywhere.

Is it wasteful, in the sense that we're generating far more random values than we'll need?  Probably, but so what?  We're talking about 10 million floating point numbers.  That's maybe 100MB of RAM occupied and a few seconds of computation time.  And if it's really too much, we'll break up the problem and do 10 successive batches of 1000 runs.

The code is just a means to an end here.  Keep it simple, and let the computer do the work.

Let's see this in action...

In [ ]:
# Set random seed for the rest of this problem
np.random.seed(47404)

In [ ]:
# Make an array of 10K "runs" of 1000 random values each, from [0,1).
randvals = np.random.random((10000,1000))
randvals  # This line here just to display the output, so you see what's happening

In [ ]:
# For each row (meaning, fixing the axis-0 value and moving "along" axis-1), perform a cumsum().
# Neat fact -- when dealing with a 2D array, you can give aggregators like cumsum() an arugment axis=k to indicate
# which axis to move along as it aggregates.
running_sums = randvals.cumsum(axis=1)

In [ ]:
# Now, we need to locate the index along each row of running sums where the value in the cumsum() first exceeds 1.0.
# Here's a tricky way to achieve this...
# Recall that boolean expressions (e.g. running_sums > 1.0) applied to arrays return an array with the same shape,
# and with every entry True or False, depending on whether the condition held there or not.  So
# running_sums > 5
# is itself an array (of True/False values), and it looks like this:
running_sums > 1.0

In [ ]:
# Great.  In each row, we'd like to know where the first True is.  How do we do that?
# Fun fact: if you tell Python to interpret True & False numerically, it interprets True as 1 and False as 0.  Viewed
# this way, each row contains only the numbers 1 and 0.  So the max value along each row happens to be 1.  Thus,
# if we ran `argmax()` along each row, it would give us the index of one of the 1s.  Which 1?
# Ah... if you look at the help notes on np.argmax here:
# https://numpy.org/doc/stable/reference/generated/numpy.argmax.html
# you'll see under "Notes" that it says: "In case of multiple occurrences of the maximum values, the indices
# corresponding to the first occurrence are returned."

# That's not exactly standard usage of argmax(), but it does what we want!
# So let's do it (argmax can also take that axis=1 argument, just like cumsum):
Nvals = np.argmax(running_sums > 1.0, axis=1)
print(Nvals)

In [ ]:
# Voila.  There are the 10K N values!

In [ ]:
# Now we need to get the avg of just the first 100, then of jus the first 1000, then of all 10,000 N values.
# Slicing to the rescue...
print(Nvals[:100].mean(), Nvals[:1000].mean(), Nvals.mean())

## Simulation 2

This problem asks us to simulate a weekly lottery.  Each week, five integers between 1 and 59 (inclusive) are drawn at random without replacement.  The first part of the problem asks us to write Python code to simulate this and to make sure we can calculate:

* $A(n)$ -- the number of times any given number $n$ is drawn  during the year; and
* $M$ -- the maximum number of times *any* number at all appears.

How should we go about simulating this lottery?


### Strategy

First, let's survey what is at our disposal.  It takes a little reading and digging, but one option is `np.random.choice()`.  Check out its documentation here:

[https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html](https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html)

It has an optional argument `replace=True/False` to indicate whether the draws should be with or without replacement.  It needs to be given the integer array to draw from as an argument, but that's straightforward -- we can build an integer array with `arange()`.

Alternately, we could just do a *permutation* of the values from 1 to 59 and slice out the first 5 from the permuted array.

This might be the easier choice, but you could use `choice()` if you prefer.

For concreteness, let's go with `np.random.permutation()`.  Simulating the whole year's lottery thus amounts to:

* making a 52 x 59 array -- each row represents a week
* applying `permutation()` along each row
* slicing out the first 5 columns to end with a 52 x 5 matrix

With a combination of a list comprehension, `random.permutation()`, and slicing, this is quick (remember, better to reuse tools than to reinvent the wheel):

In [ ]:
# Reset random seed for 2nd problem
np.random.seed(47404)

In [ ]:
# Make a 2D array by passing np.array() a list of 52 1d-arrays (which are each permutations of 1-59, sliced to only show
# the first 5 values). Remember -- arange(1,60) will start at 1 and end at 59
lottery_1yr = np.array([np.random.permutation(np.arange(1,60))[:5] for i in range(52)])
lottery_1yr  # Just here so you can see the results

Now how should our function A(n) work?  Well, we can use the whole numpy-True/False-array trick again...

Remember, something like `lottery_1yr == 42` is itself an *array* of True/False values.  The Trues are treated as 1, the Falses as 0.  So we can get a count of the number of 42s by running `sum()` on the array `lottery_1yr == 42`.

To get the number of occurrences of all the numbers from 1 to 59, use a list comprehension:

In [ ]:
num_occurrences = np.array([np.sum(lottery_1yr == i) for i in range(1,60)])
num_occurrences

And of course, to get the maximum value of *any* occurrence, just take the `max()` of that array:

In [ ]:
num_occurrences.max()

The latter half of the problem asks us to simulate many (say, $N=10,000$) versions of the 1-year lottery and count what fraction of those simulations have $A(13) \ge 17$ and $M \ge 17$.  To accomplish this, we might as well calculate $A(n)$ for all $1 \le n \le 59$ for all $N$ simulations (since we'll need all the $A(n)$ values anyway in order to get $M$ in each simulation).

How do we go about this?

We could always write a `for` loop and repeat the above 10,000 times.

Well, generalizing from our experience with Simulation 1 above, we could make a *3D array* of one-year simultations.  It's shape could be (10K, 52, 5), which you can think of as an array of ten thousand 52x5 grids, one grid for each one-year simulation.

Wasteful?  Maybe.  We'll be storing a lot of unneeded integers.  But it's not that much storage, ultimately, and it runs fairly quickly, as we'll see (maybe a few seconds to generate all 10K simulations).

Beyond that, not much needs to be changed, thanks to what we can do with the `axis=` arguments of functions like `np.sum()`.

Let's go through it step by step...

In [ ]:
# Make a 3D array by adding a list comprehension around our earlier list comprehension
N = 10000
Nlotteries = np.array([ [np.random.permutation(np.arange(1,60))[:5] for i in range(52)] for j in range(N)])
print(Nlotteries.shape)  # Just here so you can see that we've built a 3D grid as described above

In [ ]:
# Now let's proceed as before -- make a boolean array of where Nlotteries takes on each value between 1 & 59 --
# but this time, we'll give np.sum() an argument axis=(1,2).  This tells sum() to add over the last two axes but not to
# add along axis-0.  In other words, sum *within* each row, but not from row to row (since different rows represent
# runs of the simulation).
num_occurrences = np.array([np.sum(Nlotteries == i, axis=(1,2)) for i in range(1,60)])
num_occurrences.shape  # Notice that we end up with a 59x10K array -- for each number in the lottery, we have
                       # a number of occurrences in each of the ten-thousand simulations

In [ ]:
# To get the 10K A(13) values, we want row 12 (b/c the array is zero-indexed).  To find how many of those 10K entries
# are >= 17, we can use our usual boolean array trick:
# NOTE: num_occurrences[12] is a 1D array with 10K elements
A13 = num_occurrences[12] # Defining this variable for ease of reference
A13 >= 17  # This is a boolean array, and it appears to be mostly filled with False...

In [ ]:
# Let's see if, in fact, it's always False.  It is always False.
np.any(A13 >= 17)

So this makes $P(A(13) >= 17)$ appear to be extremely close to zero.  We'll need a much larger $N$ to resolve *how* close to zero.  And just to validate that we don't have a code glitch, note that if we ask instead how often $A(13) >= 5$, we get around 45% of the time.  For $A(13) >= 10$, we're down to about 1% of the time.

In [ ]:
print(np.sum(A13 >= 5))
print(np.sum(A13 >= 10))

To get all 10K $M$ values, we can apply `max()` along `axis=0` of `num_occurrences`:

In [ ]:
NM = num_occurrences.max(axis=0)
NM.shape # This is a length-10K 1D array of M values (one for each simulation), as it should be

To wrap up, we can ask what fraction of the 10K M values were $\ge 17$.  But again, the answer appears too close to zero to discern with only 10K samples.

In [ ]:
np.any(NM >= 17)

It may be necessary to redo the latter half of the problem with a much higher value of $N$ (perhaps $N \ge 10^6$) in order to estimate these near-zero probabilities.  If either the memory requirements get too severe or the code runs too long, you can separately run 100 versions of the $N=10^4$ simulations and record whether any give a nonzero value for the fraction with either $A(13) >=17$ or $M >= 17$.

All of this is left as an exercise.

#### Using NumPy dtypes to save space

In the event that you need a much larger $N$ than we've discussed to get a handle on this probability via simulation, it's worth noting that there's a way to make each 59x5 grid of one-year lottery results occupy less memory.  Elements in a NumPy array must all be of the same data type, but the data type doesn't have to be a native Python data type.  NumPy understands C-level data types, which are more nuanced.  For instance, instead of just `int`s, NumPy can accommodate signed or unsigned integers and even allocate different amounts of space, i.e. a different number of available digits, to store that number.

A useful list of NumPy's types is at [https://numpy.org/devdocs/user/basics.types.html](https://numpy.org/devdocs/user/basics.types.html).

Since we are only dealing positive integers under 60, we can store them easily as 8-bit (i.e. 1-byte) unsigned integer (that can accommodate numbers up to 255). The link above tells us that data type is called `np.uint8`.  By default, NumPy stores integers using 64 bits (8 bytes), so this represents a factor of 8 savings.  That might make the different between having enough memory on your laptop to hold the array or not, depending on how large we end up needing to make $N$.

Note that our code doesn't ever perform any arithmetic on these integers -- any summing is performed only on the True/False values interpreted as 1s or 0s -- so we don't need to worry about having extra "buffer" storage in case an intermediate sum became larger than 255.

To tell NumPy to store the integers as `np.uint8` rather than as the default `np.int64`, we pass the argument `dtype=np.uint8` to the array that first generates these integers.  For us, that was handled by `arange`, so we would modify the line where we build our $N$ one-year simulations as follows:
```python
# Make a 3D array by adding a list comprehension around our earlier list comprehension.
# Give arange() a dtype=np.uint8 argument to use less storage.
N = 10000
Nlotteries = np.array([ [np.random.permutation(np.arange(1,60, dtype=np.uint8))[:5] for i in range(52)] for j in range(N)])

```

## Moral of the story

* Before you write a lot of code, see if there are existing tools and constructs you can cobble together to do what you want, especially in a course where the coding is means and not the ends.
* Proceed incrementally -- don't try to do the whole enchilada at once until you have figured out the mechanics of doing things right with smaller samples and a smaller number of simulations runs